In [1]:
!python3 -m pip install -e ../.

Looking in indexes: https://pypi.org/simple, https://pypi.org/simple
Obtaining file:///Users/nelenj/Documents/projects/helmet
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for helmet (pyproject.toml) ... done
  Created wheel for helmet: filename=helmet-0.1.2-py3-none-any.whl size=1745 sha256=5e1e92d5fe06518cb5b9d96e61e0c65d7a11e9b2a1719b1e9ec8338fbbe3bbb0
  Stored in directory: /private/var/folders/f1/st6fs1q547zbfmzy6jr5b0km0000gp/T/pip-ephem-wheel-cache-6ssn5rru/wheels/b5/be/d1/aaf642ac2f1062e27dc45a2559b93e86459c128f1f8ea5cea1
Successfully built helmet
  Attempting uninstall: helmet
    Found existing installation: helmet 0.1.2
    Uninstalling helmet-0.1.2:
      Successfully uninstalled helmet-0.1.2


In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# This to make sure that the changes in the module are reloaded
%reload_ext autoreload

In [7]:
# Platform configs
url = "https://api.jeroennelen.nl"
project_id = "662181a8e92e593fc2882536"

project_setup = {
    "platform_url": url,
    "project_id": project_id
}

In [8]:
# checkpoint = "mlabonne/Gemmalpaca-2B"
# "meta-llama/Meta-Llama-3-8B"
device = "cpu"
# checkpoint = "bigscience/bloom-1b7"
checkpoint = "bigscience/bloom-560m"

model_setup = {
    "checkpoint": checkpoint,
    "model_type": "dec",
    "embeddings": "transformer.word_embeddings"
}

run_config = {
    "device": device,
    "max_new_tokens": 100,
    # "temperature": 1, # Not
    # e that if you add this, then you need to add "do_sample": True
    # "do_sample": True
}

In [9]:
import helmet

/Users/nelenj/miniconda3/envs/helmet-installable/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
model = helmet.from_pretrained(project_setup=project_setup, model_setup=model_setup, run_config=run_config)

updates will be sent to https://api.jeroennelen.nl
setting up model with config {'checkpoint': 'bigscience/bloom-560m', 'model_type': 'dec', 'embeddings': 'transformer.word_embeddings'}
model loaded


In [14]:
predict_config = {
    "max_new_tokens": 20,
    "generate_explanations": False,
}

prompt = "test"
ground_truth = "test a lot of time"
output = model.predict(prompt, groundtruth=ground_truth, **predict_config)
output

Predicted output: .test.test.test.test.test.test.test.test.test.test
updated app, result:  {'acknowledged': True, 'insertedId': '6627a9539f224fc8058a69e0'}


'.test.test.test.test.test.test.test.test.test.test'

In [ ]:
# !pip install huggingface_hub ipywidgets -q

In [ ]:
# This might be needed to load the model
# from huggingface_hub import login
# login()

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Helsinki-NLP/opus-100", name="en-it", split='test[:20]')

In [ ]:
dataset[2]

In [ ]:
# prompt = dataset["validation"][1]
# # print(prompt)
# # prompt = prompt["question"]

In [ ]:
source_lang = "en"
target_lang = "it"
prefix = "Translate English to Italian: "

def prompt_builder(sample):
    return prefix + sample['translation'][source_lang]


prompt = prompt_builder(dataset[2])
prompt

In [ ]:
predict_config = {
    "max_new_tokens": 20,
    "generate_explanations": False,
}

output = model.predict(prompt, **predict_config)
output

In [ ]:
for i in range(20):
    prompt = prompt_builder(dataset[i])
    result = model.predict(prompt, **predict_config)
    print(result)

In [ ]:
model.saliency_explainer("6627a1879f224fc8058a69c9")